## Imports and Setup

In [2]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
pd.set_option('display.max_columns', 500)

## Data Import 

In [30]:
data = pd.read_csv('PredictionData.csv')
data_invert = pd.read_csv('PredictionDataInvert.csv')
data['Conference_team_one'] = data['Conference_team_one'].astype('category')
data['Conference_team_two'] = data['Conference_team_two'].astype('category')

data_invert['Conference_team_one'] = data_invert['Conference_team_one'].astype('category')
data_invert['Conference_team_two'] = data_invert['Conference_team_two'].astype('category')

cat_columns = data.select_dtypes(['category']).columns
data[cat_columns] = data[cat_columns].apply(lambda x: x.cat.codes)

cat_columns = data_invert.select_dtypes(['category']).columns
data_invert[cat_columns] = data_invert[cat_columns].apply(lambda x: x.cat.codes)

In [31]:
data.head()

,Seed_team_one,Team_team_one,Conference_team_one,AdjEM_team_one,AdjO_team_one,AdjD_team_one,AdjT_team_one,Luck_team_one,Opp AdjEM_team_one,OppO_team_one,OppD_team_one,NCSOS AdjEM_team_one,W_team_one,L_team_one,ast_pct_team_one,blk_pct_team_one,efg_pct_team_one,fg3a_per_fga_pct_team_one,ft_rate_team_one,fta_per_fga_pct_team_one,losses_conf_team_one,losses_home_team_one,losses_visitor_team_one,opp_pts_team_one,pts_team_one,sos_team_one,srs_team_one,tov_pct_team_one,trb_pct_team_one,ts_pct_team_one,wins_conf_team_one,wins_home_team_one,wins_visitor_team_one,def_rtg_team_one,opp_fta_per_fga_pct_team_one,opp_fg3a_per_fga_pct_team_one,opp_ts_pct_team_one,opp_trb_pct_team_one,opp_ast_pct_team_one,opp_stl_pct_team_one,opp_blk_pct_team_one,opp_efg_pct_team_one,opp_tov_pct_team_one,opp_orb_pct_team_one,opp_ft_rate_team_one,num_JR_team_one,num_FR_team_one,num_SR_team_one,num_SO_team_one,num_G_team_one,num_F_team_one,ast_per_g_1_team_one,ast_per_g_2_team_one,ast_per_g_3_team_one,ast_per_g_4_team_one,ast_per_g_5_team_one,ast_per_g_6_team_one,ast_per_g_7_team_one,ast_per_g_8_team_one,ast_per_g_9_team_one,ast_per_g_10_team_one,ast_per_g_11_team_one,ast_per_g_12_team_one,ast_per_g_13_team_one,blk_per_g_1_team_one,blk_per_g_2_team_one,blk_per_g_3_team_one,blk_per_g_4_team_one,blk_per_g_5_team_one,blk_per_g_6_team_one,blk_per_g_7_team_one,blk_per_g_8_team_one,blk_per_g_9_team_one,blk_per_g_10_team_one,blk_per_g_11_team_one,blk_per_g_12_team_one,blk_per_g_13_team_one,drb_per_g_1_team_one,drb_per_g_2_team_one,drb_per_g_3_team_one,drb_per_g_4_team_one,drb_per_g_5_team_one,drb_per_g_6_team_one,drb_per_g_7_team_one,drb_per_g_8_team_one,drb_per_g_9_team_one,drb_per_g_10_team_one,drb_per_g_11_team_one,drb_per_g_12_team_one,drb_per_g_13_team_one,fg2_pct_1_team_one,fg2_pct_2_team_one,fg2_pct_3_team_one,fg2_pct_4_team_one,fg2_pct_5_team_one,fg2_pct_6_team_one,fg2_pct_7_team_one,fg2_pct_8_team_one,fg2_pct_9_team_one,fg2_pct_10_team_one,fg2_pct_11_team_one,fg2_pct_12_team_one,fg2_pct_13_team_one,fg2_per_g_1_team_one,fg2_per_g_2_team_one,fg2_per_g_3_team_one,fg2_per_g_4_team_one,fg2_per_g_5_team_one,fg2_per_g_6_team_one,fg2_per_g_7_team_one,fg2_per_g_8_team_one,fg2_per_g_9_team_one,fg2_per_g_10_team_one,fg2_per_g_11_team_one,fg2_per_g_12_team_one,fg2_per_g_13_team_one,fg2a_per_g_1_team_one,fg2a_per_g_2_team_one,fg2a_per_g_3_team_one,fg2a_per_g_4_team_one,fg2a_per_g_5_team_one,fg2a_per_g_6_team_one,fg2a_per_g_7_team_one,fg2a_per_g_8_team_one,fg2a_per_g_9_team_one,fg2a_per_g_10_team_one,fg2a_per_g_11_team_one,fg2a_per_g_12_team_one,fg2a_per_g_13_team_one,fg3_pct_1_team_one,fg3_pct_2_team_one,fg3_pct_3_team_one,fg3_pct_4_team_one,fg3_pct_5_team_one,fg3_pct_6_team_one,fg3_pct_7_team_one,fg3_pct_8_team_one,fg3_pct_9_team_one,fg3_pct_10_team_one,fg3_pct_11_team_one,fg3_pct_12_team_one,fg3_pct_13_team_one,fg3_per_g_1_team_one,fg3_per_g_2_team_one,fg3_per_g_3_team_one,fg3_per_g_4_team_one,fg3_per_g_5_team_one,fg3_per_g_6_team_one,fg3_per_g_7_team_one,fg3_per_g_8_team_one,fg3_per_g_9_team_one,fg3_per_g_10_team_one,fg3_per_g_11_team_one,fg3_per_g_12_team_one,fg3_per_g_13_team_one,fg3a_per_g_1_team_one,fg3a_per_g_2_team_one,fg3a_per_g_3_team_one,fg3a_per_g_4_team_one,fg3a_per_g_5_team_one,fg3a_per_g_6_team_one,fg3a_per_g_7_team_one,fg3a_per_g_8_team_one,fg3a_per_g_9_team_one,fg3a_per_g_10_team_one,fg3a_per_g_11_team_one,fg3a_per_g_12_team_one,fg3a_per_g_13_team_one,fg_pct_1_team_one,fg_pct_2_team_one,fg_pct_3_team_one,fg_pct_4_team_one,fg_pct_5_team_one,fg_pct_6_team_one,fg_pct_7_team_one,fg_pct_8_team_one,fg_pct_9_team_one,fg_pct_10_team_one,fg_pct_11_team_one,fg_pct_12_team_one,fg_pct_13_team_one,fg_per_g_1_team_one,fg_per_g_2_team_one,fg_per_g_3_team_one,fg_per_g_4_team_one,fg_per_g_5_team_one,fg_per_g_6_team_one,fg_per_g_7_team_one,fg_per_g_8_team_one,fg_per_g_9_team_one,fg_per_g_10_team_one,fg_per_g_11_team_one,fg_per_g_12_team_one,fg_per_g_13_team_one,fga_per_g_1_team_one,fga_per_g_2_team_one,fga_per_g_3_team_one,fga_per_g_4_team_one,fga_per_g_5_team_one,fga_per

In [32]:
labels = np.array(data['result'])
labels = np.reshape(labels, newshape=(labels.shape[0], 1))
scores = data[['score_one', 'score_two']].to_numpy()
labels_invert = np.array(data_invert['result'])
labels_invert = np.reshape(labels_invert, newshape=(labels_invert.shape[0], 1))
scores_invert = data_invert[['score_one', 'score_two']].to_numpy()

data = data.drop(columns=['result', 'score_one', 'score_two'])
data_invert = data_invert.drop(columns=['result', 'score_one', 'score_two'])

In [34]:
cols_to_drop = ['Team_team_one', 'Team_team_two', 'year_team_one', 'year_team_two']

cols_not = [col for col in data.columns if col not in cols_to_drop]

data = data[cols_to_drop + cols_not]
data_invert = data_invert[cols_to_drop + cols_not]

data_np = data.drop(columns=cols_to_drop).to_numpy()

data_invert_np = data_invert.drop(columns=cols_to_drop).to_numpy()

## Tensorflow Experiment

In [63]:
num_features = data_np.shape[1]
num_neurons = num_features

In [64]:
x = tf.placeholder(dtype=tf.float32, shape=(None, num_features))
y = tf.placeholder(dtype=tf.float32, shape=(None, 1))

b_hidden = tf.Variable(tf.zeros([num_neurons]))
W_hidden = tf.Variable(tf.random_normal([num_features, num_neurons]))

z_hidden = tf.add(tf.matmul(x, W_hidden), b_hidden)
a_hidden = tf.nn.relu(z_hidden) # shape is (910)

b_out = tf.Variable(tf.zeros([1]))
W_out = tf.Variable(tf.random_normal([num_features, 1]))

z_out = tf.add(tf.matmul(a_hidden, W_out), b_out)
# a_out = tf.sigmoid(z_out)

In [65]:
training_epochs = 3000
learning_rate = 0.01

cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=z_out)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

predicted = tf.nn.sigmoid(z_out)
correct_pred = tf.equal(tf.round(predicted), y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [66]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(training_epochs + 1):
        sess.run(optimizer, feed_dict={x: data_np, y: labels})
        loss, _, acc = sess.run([cost, optimizer, accuracy], feed_dict={x: data_np, y: labels})
        if step % 500 == 0:
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))

Step:     0	Loss: 84470.352	Acc: 29.59%
Step:   500	Loss: 96.448	Acc: 89.84%
Step:  1000	Loss: 19.164	Acc: 95.95%
Step:  1500	Loss: 8.098	Acc: 97.57%
Step:  2000	Loss: 0.000	Acc: 100.00%
Step:  2500	Loss: 0.000	Acc: 100.00%
Step:  3000	Loss: 0.000	Acc: 100.00%


## Define Tensorflow Models and Data

In [55]:
def build_model(num_features, num_hidden_layers=3, learning_rate=0.001):
    model = tf.keras.Sequential()
    for _ in range(num_hidden_layers):
        model.add(layers.Dense(units=num_features, activation='relu'))
    
    model.add(layers.Dense(units=1, activation='sigmoid'))
    
    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate),
                  loss='binary_crossentropy', 
                  metrics=['binary_crossentropy', 'accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
    
    return model

def get_callbacks(scaler=True, split=True):
    return [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss' if split else 'loss', patience=20 if scaler else 100)
    ]

In [56]:
class Data:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def split(self, split=False):
        if split:
            X_train, X_test, y_train, y_test = train_test_split(self.data, self.labels)
        else:
            X_train, X_test, y_train, y_test = self.data, None, self.labels, None
        
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
        return self
        
    def clone(self):
        data_new = Data(self.data, self.labels)
        if hasattr(self, 'X_train'):
            data_new.X_train = self.X_train
            data_new.X_test = self.X_test
            data_new.y_train = self.y_train
            data_new.y_test = self.y_test
        if hasattr(self, 'scaler'):
            data_new.scaler = self.scaler
        if hasattr(self, 'model'):
            data_new.model = self.model
        return data_new
    
    def add_scaler(self, scaler):
        self.scaler = scaler
    
    def fit_scaler(self):
        if not self.scaler:
            return
        self.scaler = self.scaler.fit(self.X_train)
        self.X_train = self.scaler.transform(self.X_train)
        if self.X_test is not None:
            self.X_test = self.scaler.transform(self.X_test)
        self.data = self.scaler.transform(self.data)
    
    def add_model(self, model):
        self.model = model
        
    def fit_model(self, verbose=3):
        if verbose == 0:
            print (f'Fitting model {self.label}')
        if self.scaler:
            if self.X_test is None and self.y_test is None:
                self.history = self.model.fit(x=self.X_train, 
                                              y=self.y_train, 
                                              batch_size=100, 
                                              callbacks=get_callbacks(True, False), 
                                              epochs=100, 
                                             verbose=verbose)
            else:
                self.history = self.model.fit(x=self.X_train, 
                                              y=self.y_train, 
                                              batch_size=100,
                                              validation_data=(self.X_test, self.y_test), 
                                              callbacks=get_callbacks(True, True), 
                                              epochs=2000, 
                                             verbose=verbose)
        else:
            if self.X_test is None and self.y_test is None:
                self.history = self.model.fit(x=self.X_train,
                                             y=self.y_train, 
                                             batch_size=100,
                                             callbacks=get_callbacks(False, False),
                                             epochs=100, 
                                             verbose=verbose)
            else:
                self.history = self.model.fit(x=self.X_train, 
                                             y=self.y_train,
                                             batch_size=100,
                                             validation_data=(self.X_test, self.y_test),
                                             callbacks=get_callbacks(False, True),
                                             epochs=2000, 
                                             verbose=verbose)
        if verbose == 0:
            print ('Done!')
        return self.history
    
    def predict_classes(self):
        self.predictions = self.model.predict_classes(x=self.data)
    
    def __str__(self):
        return f'{self.X_train.shape}, {self.X_test}, {self.y_train.shape}, {self.y_test}'

## Define Tensorflow Possibilities

In [57]:
data_obj = Data(data_np, labels)
data_invert_obj = Data(data_invert_np, labels_invert)

datas = [data_obj, data_invert_obj]

In [58]:
possibilities = []
for poss in datas:
    for val in [True, False]:
        possibilities.append(poss.clone().split(val))


label_models = ['Normal Split', 'Normal Non-Split', 'Invert Split', 'Invert Non-Split']
for poss, label in zip(possibilities, label_models):
    poss.label = label

In [59]:
possibilities_min_max = []
possibilities_standard = []

for poss in possibilities:
    poss.add_scaler(None)

for poss in possibilities:
    min_max_poss = poss.clone()
    min_max_poss.add_scaler(MinMaxScaler())
    min_max_poss.fit_scaler()
    min_max_poss.label = poss.label + ' Min Max'
    possibilities_min_max.append(min_max_poss)
    
    standard_poss = poss.clone()
    standard_poss.add_scaler(StandardScaler())
    standard_poss.fit_scaler()
    standard_poss.label = poss.label + ' Standard'
    possibilities_standard.append(standard_poss)


possibilities = possibilities + possibilities_min_max + possibilities_standard

## Add Models and Fit

In [60]:
for poss in possibilities:
    poss.add_model(build_model(poss.X_train.shape[1]))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [61]:
# for poss in possibilities:
#     poss.fit_model(0)    

## Plot Training of Models

In [62]:
num_plots = len(possibilities)
fig = plt.figure(figsize=(10, 20))
num_y = num_plots // 2

for idx, poss in enumerate(possibilities):
    history = poss.history
    plt.subplot(num_y, 2, idx+1)
    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['acc'], label='Accuracy')
    
    if history.history.get('val_loss') is not None:
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.plot(history.history['val_acc'], label='Validation Accuracy')
    
    plt.title(f'Training History ({poss.label})')
    plt.legend()
    plt.ylim((0, 1))

plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

## Evaluate TF Models by Observing Prediction for UVA-UMBC game and viewing Classification Reports and Confusion Matrices

In [8]:
row = data[(data['Team_team_one'] == 'Virginia') & (data['year_team_one'] == 2018)]
row_unc = data[(data['Team_team_two'] == 'North Carolina') & (data['year_team_one'] == 2017) & (data['Team_team_one'] == 'Gonzaga')]

row = row.drop(columns=cols_to_drop).to_numpy()
row_unc = row_unc.drop(columns=cols_to_drop).to_numpy()

In [9]:
print (row.shape)
print (row_unc.shape)
print (data_np.shape)

(1, 910)
(1, 910)
(1112, 910)


In [83]:
for poss in possibilities:
    prediction = poss.model.predict_proba(x=row if not poss.scaler else poss.scaler.transform(row))
    print (poss.label)
    print (prediction)

NameError: name 'possibilities' is not defined

In [21]:
for poss in possibilities:
    poss.predict_classes()

In [23]:
for poss in possibilities:
    print (poss.label)
    print (classification_report(poss.labels, poss.predictions))

Normal Split
              precision    recall  f1-score   support

           0       0.88      0.74      0.80       329
           1       0.90      0.96      0.93       783

    accuracy                           0.89      1112
   macro avg       0.89      0.85      0.86      1112
weighted avg       0.89      0.89      0.89      1112

Normal Non-Split
              precision    recall  f1-score   support

           0       0.75      0.67      0.71       329
           1       0.87      0.91      0.89       783

    accuracy                           0.84      1112
   macro avg       0.81      0.79      0.80      1112
weighted avg       0.83      0.84      0.83      1112

Invert Split
              precision    recall  f1-score   support

           0       0.96      0.74      0.83      1112
           1       0.79      0.97      0.87      1112

    accuracy                           0.85      2224
   macro avg       0.87      0.85      0.85      2224
weighted avg       0.87      0.

In [24]:
for poss in possibilities:
    print (poss.label)
    print (confusion_matrix(poss.labels, poss.predictions))

Normal Split
[[242  87]
 [ 34 749]]
Normal Non-Split
[[220 109]
 [ 73 710]]
Invert Split
[[ 820  292]
 [  36 1076]]
Invert Non-Split
[[ 762  350]
 [  68 1044]]
Normal Split Min Max
[[286  43]
 [ 64 719]]
Normal Non-Split Min Max
[[329   0]
 [  0 783]]
Invert Split Min Max
[[ 873  239]
 [  27 1085]]
Invert Non-Split Min Max
[[1101   11]
 [  55 1057]]
Normal Split Standard
[[287  42]
 [ 39 744]]
Normal Non-Split Standard
[[329   0]
 [  0 783]]
Invert Split Standard
[[1023   89]
 [  48 1064]]
Invert Non-Split Standard
[[1112    0]
 [   0 1112]]


## Now, let's examine some SKLearn Algorithms

In [10]:
# SKLearn Algorithms
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, ElasticNet, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neural_network import MLPClassifier

In [11]:
labels_sklearn = labels.flatten()
labels_invert_sklearn = labels_invert.flatten()

print (labels_sklearn.shape)

sklearn_space = [(data_np, labels_sklearn, 'Data'), (data_invert_np, labels_invert_sklearn, 'Data Invert')]

sklearn_score_space = [(data_np, scores, 'Data'), (data_invert_np, scores_invert, 'Data Invert')]

(1112,)


### Random Forest

In [62]:
%%time

## Random Forest
param_space_random_forest = {
    'rf__bootstrap' : [True, False],
    'rf__n_estimators' : [100, 1000]
}

random_forests = []
for poss in sklearn_space:
    random_forest_min_max = RandomForestClassifier(n_jobs=-1)
    pipe = Pipeline(steps=[('scale', MinMaxScaler()), ('rf', random_forest_min_max)])
    
    grid_search_forest = GridSearchCV(estimator=pipe, param_grid=param_space_random_forest, cv=StratifiedKFold(n_splits=10))
    
    random_forest_standard = RandomForestClassifier(n_jobs=-1)
    pipe_standard = Pipeline(steps=[('scale', StandardScaler()), ('rf', random_forest_standard)])
    
    grid_search_forest_standard = GridSearchCV(estimator=pipe_standard, param_grid=param_space_random_forest, cv=StratifiedKFold(n_splits=10))
    
    random_forests.append((poss[0], poss[1], grid_search_forest, poss[2] + ' Min Max'))
    random_forests.append((poss[0], poss[1], grid_search_forest_standard, poss[2] + ' Standard'))

for X, y, forest, _ in random_forests:
    forest.fit(X, y)
    
for X, y, forest, label in random_forests:
    print (label)
    print ('{:.4f}'.format(forest.score(X, y)))

Data Min Max
1.0000
Data Standard
1.0000
Data Invert Min Max
1.0000
Data Invert Standard
1.0000
CPU times: user 3min 11s, sys: 24.5 s, total: 3min 36s
Wall time: 10min 39s


In [63]:
for _, _, forest, label in random_forests:
    print (label)
    print (forest.best_params_)

Data Min Max
{'rf__bootstrap': False, 'rf__n_estimators': 100}
Data Standard
{'rf__bootstrap': False, 'rf__n_estimators': 100}
Data Invert Min Max
{'rf__bootstrap': False, 'rf__n_estimators': 1000}
Data Invert Standard
{'rf__bootstrap': False, 'rf__n_estimators': 1000}


In [64]:
for _, _, forest, label in random_forests:
    feature_importances = forest.best_estimator_.named_steps['rf'].feature_importances_
    feature_importances = [(idx, val) for idx, val in enumerate(feature_importances)]
    feature_importances.sort(key=lambda k: k[1], reverse=True)
    feature_importances = [idx+len(cols_to_drop) for idx, _ in feature_importances[:10]]
    print (label)
    for idx, col in enumerate(data.columns[feature_importances]):
        print (f'{idx+1}. {col}')

Data Min Max
1. AdjEM_team_two
2. pts_team_two
3. W_team_two
4. pts_team_one
5. AdjEM_team_one
6. W_team_one
7. AdjD_team_two
8. opp_pts_team_one
9. srs_team_two
10. srs_team_one
Data Standard
1. srs_team_two
2. pts_team_two
3. W_team_two
4. pts_team_one
5. AdjEM_team_two
6. AdjEM_team_one
7. W_team_one
8. OppD_team_two
9. opp_pts_team_one
10. Opp AdjEM_team_one
Data Invert Min Max
1. AdjEM_team_two
2. AdjEM_team_one
3. W_team_one
4. W_team_two
5. srs_team_one
6. pts_team_one
7. pts_team_two
8. srs_team_two
9. Opp AdjEM_team_two
10. Opp AdjEM_team_one
Data Invert Standard
1. AdjEM_team_one
2. AdjEM_team_two
3. W_team_one
4. W_team_two
5. srs_team_two
6. pts_team_one
7. srs_team_one
8. pts_team_two
9. Opp AdjEM_team_two
10. Opp AdjEM_team_one


In [65]:
for X, y, forest, label in random_forests:
    print (label)
    y_pred = forest.predict(X)
    print (confusion_matrix(y, y_pred))

Data Min Max
[[329   0]
 [  0 783]]
Data Standard
[[329   0]
 [  0 783]]
Data Invert Min Max
[[1112    0]
 [   0 1112]]
Data Invert Standard
[[1112    0]
 [   0 1112]]


In [66]:
for _, _, forest, label in random_forests:
    print (label)
    print (forest.predict(row))
    print (forest.predict(row_unc))

Data Min Max
[-1]
[-1]
Data Standard
[-1]
[-1]
Data Invert Min Max
[-1]
[-1]
Data Invert Standard
[-1]
[-1]


In [77]:
for _, _, forest, label in random_forests:
    print (label)
    scaled_row = forest.best_estimator_.named_steps['scale'].transform(row_unc)
    print (forest.best_estimator_.named_steps['rf'].predict(scaled_row))

Data Min Max
[-1]
Data Standard
[-1]
Data Invert Min Max
[-1]
Data Invert Standard
[-1]


### Logistic Regression

In [36]:
%%time

## Logistic Regrerssion
param_space_log_reg = {
    'lr__C' : [10**i for i in range(-3, 2)],
    'lr__penalty' : ['l1']
}

log_regs = []
for poss in sklearn_space:
    logreg_min_max = LogisticRegression(max_iter=int(1e3), solver='saga', n_jobs=-1)
    pipe = Pipeline(steps=[('scale', MinMaxScaler()), ('lr', logreg_min_max)])
    
    curr_labels = poss[1]
    curr_labels[curr_labels==0] = -1
    
    grid_search_logreg = GridSearchCV(estimator=pipe, param_grid=param_space_log_reg, cv=StratifiedKFold(n_splits=10))
    
    logreg_standard = LogisticRegression(max_iter=int(1e3), solver='saga', n_jobs=-1)
    pipe_standard = Pipeline(steps=[('scale', StandardScaler()), ('lr', logreg_standard)])
    
    grid_search_logreg_standard = GridSearchCV(estimator=pipe_standard, param_grid=param_space_log_reg, cv=StratifiedKFold(n_splits=10))
    
    log_regs.append((poss[0], curr_labels, grid_search_logreg, poss[2] + ' Min Max'))
    log_regs.append((poss[0], curr_labels, grid_search_logreg_standard, poss[2] + ' Standard'))
    
for X, y, logreg, _ in log_regs:
    logreg.fit(X, y)
    
for X, y, logreg, label in log_regs:
    print (label)
    print ('{:.4f}'.format(logreg.score(X, y)))

/mnt/c/Users/tas12/Documents/Projects/BracketProject/bracketenv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/mnt/c/Users/tas12/Documents/Projects/BracketProject/bracketenv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/mnt/c/Users/tas12/Documents/Projects/BracketProject/bracketenv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Data Min Max
0.9451
Data Standard
0.9056
Data Invert Min Max
0.9155
Data Invert Standard
0.9101
CPU times: user 2min 3s, sys: 1min 6s, total: 3min 10s
Wall time: 52min


/mnt/c/Users/tas12/Documents/Projects/BracketProject/bracketenv/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [38]:
for _, _, logreg, label in log_regs:
    print (label)
    print (logreg.best_params_)

Data Min Max
{'lr__C': 1, 'lr__penalty': 'l1'}
Data Standard
{'lr__C': 0.1, 'lr__penalty': 'l1'}
Data Invert Min Max
{'lr__C': 1, 'lr__penalty': 'l1'}
Data Invert Standard
{'lr__C': 0.1, 'lr__penalty': 'l1'}


In [92]:
for _, _, logreg, label in log_regs:
    print (label)
    coefs = logreg.best_estimator_.named_steps['lr'].coef_[0]
    coefs = [(idx, coef) for idx, coef in enumerate(coefs)]
    coefs.sort(key=lambda k: k[1], reverse=True)
    coef_idxs = [idx for idx, _ in coefs]
    
    top_coefs = coef_idxs[:5]
    bottom_coefs = coef_idxs[-5:]
    print (list(data.columns[top_coefs]))
    print (list(data.columns[bottom_coefs]))

# coefs = [(idx, coef) for idx, coef in enumerate(logreg.coef_[0])]
# coefs.sort(key=lambda k : k[1], reverse=True)

# coef_idxs = [idx for idx, _ in coefs]
# for idx, col in enumerate(data.columns[coef_idxs]):
#     print (f'{col}: {coefs[idx][1]}')

Data Min Max
['opp_pts_team_one', 'losses_visitor_team_one', 'NCSOS AdjEM_team_one', 'Conference_team_two', 'Conference_team_one']
['Seed_team_two', 'AdjO_team_one', 'losses_conf_team_two', 'losses_home_team_two', 'OppO_team_two']
Data Standard
['losses_visitor_team_one', 'opp_pts_team_one', 'NCSOS AdjEM_team_one', 'Conference_team_two', 'Conference_team_one']
['AdjO_team_one', 'Seed_team_two', 'losses_conf_team_two', 'losses_home_team_two', 'OppO_team_two']
Data Invert Min Max
['NCSOS AdjEM_team_one', 'opp_pts_team_one', 'losses_visitor_team_one', 'Conference_team_two', 'Conference_team_one']
['Seed_team_two', 'AdjO_team_one', 'losses_conf_team_two', 'losses_home_team_two', 'OppO_team_two']
Data Invert Standard
['losses_visitor_team_one', 'NCSOS AdjEM_team_one', 'opp_pts_team_one', 'Conference_team_two', 'Conference_team_one']
['Seed_team_two', 'AdjO_team_one', 'losses_home_team_two', 'OppO_team_two', 'losses_conf_team_two']


In [39]:
for X, y, logreg, label in log_regs:
    print (label)
    y_pred = logreg.predict(X)
    print (confusion_matrix(y, y_pred))

Data Min Max
[[286  43]
 [ 18 765]]
Data Standard
[[257  72]
 [ 33 750]]
Data Invert Min Max
[[1018   94]
 [  94 1018]]
Data Invert Standard
[[1012  100]
 [ 100 1012]]


### Gaussian Naive Bayes

In [41]:
%%time

param_space_gaussian = {
    
}

gaussians = []
for poss in sklearn_space:
    gaussian_min_max = GaussianNB()
    pipe = Pipeline(steps=[('scale', MinMaxScaler()), ('gauss', gaussian_min_max)])
    
    grid_search_gaussian = GridSearchCV(estimator=pipe, param_grid=param_space_gaussian, cv=StratifiedKFold(n_splits=10))
    
    gaussian_standard = GaussianNB()
    pipe_standard = Pipeline(steps=[('scale', StandardScaler()), ('gauss', gaussian_standard)])
    
    grid_search_gaussian_standard = GridSearchCV(estimator=pipe_standard, param_grid=param_space_gaussian, cv=StratifiedKFold(n_splits=10))
    
    gaussians.append((poss[0], poss[1], grid_search_gaussian, poss[2] + ' Min Max'))
    gaussians.append((poss[0], poss[1], grid_search_gaussian_standard, poss[2] + ' Standard'))
    
for X, y, gaussian, _ in gaussians:
    gaussian.fit(X, y)
    
for X, y, gaussian, label in gaussians:
    print (label)
    print ('{:.4f}'.format(gaussian.score(X, y)))

Data Min Max
0.4263
Data Standard
0.3939
Data Invert Min Max
0.7203
Data Invert Standard
0.7203
CPU times: user 1.59 s, sys: 93.8 ms, total: 1.69 s
Wall time: 1.7 s


### Perceptron

In [42]:
%%time

param_space_perceptron = {
    'perceptron__penalty' : [None, 'l2', 'l1', 'elasticnet'],
    'perceptron__alpha' : [10**-3]
}

perceptrons = []
for poss in sklearn_space:
    perceptron_min_max = Perceptron()
    pipe = Pipeline(steps=[('scale', MinMaxScaler()), ('perceptron', perceptron_min_max)])
    
    grid_search_perceptron = GridSearchCV(estimator=pipe, param_grid=param_space_perceptron, cv=StratifiedKFold(n_splits=10))
    
    perceptron_standard = Perceptron()
    pipe_standard = Pipeline(steps=[('scale', StandardScaler()), ('perceptron', perceptron_standard)])
    
    grid_search_perceptron_standard = GridSearchCV(estimator=pipe_standard, param_grid=param_space_perceptron, cv=StratifiedKFold(n_splits=10))
    
    perceptrons.append((poss[0], poss[1], grid_search_perceptron, poss[2] + ' Min Max'))
    perceptrons.append((poss[0], poss[1], grid_search_perceptron_standard, poss[2] + ' Standard'))
    
for X, y, perceptron, _ in perceptrons:
    perceptron.fit(X, y)
    
for X, y, perceptron, label in perceptrons:
    print (label)
    print ('{:.4f}'.format(perceptron.score(X, y)))

Data Min Max
0.9254
Data Standard
0.9559
Data Invert Min Max
0.7455
Data Invert Standard
0.9096
CPU times: user 37.4 s, sys: 18.5 s, total: 55.8 s
Wall time: 26.1 s


In [43]:
for _, _, perceptron, label in perceptrons:
    print (label)
    print (perceptron.best_params_)

Data Min Max
{'perceptron__alpha': 0.001, 'perceptron__penalty': 'l1'}
Data Standard
{'perceptron__alpha': 0.001, 'perceptron__penalty': 'l1'}
Data Invert Min Max
{'perceptron__alpha': 0.001, 'perceptron__penalty': 'l1'}
Data Invert Standard
{'perceptron__alpha': 0.001, 'perceptron__penalty': 'l1'}


### MLPClassifier

In [136]:
%%time

## Logistic Regrerssion
param_space_nn = {
#     'lr__C' : [10**i for i in range(-5, 2)]
}

nns = []
for poss in sklearn_space:
    num_features = poss[0].shape[1]
    nn_min_max = MLPClassifier(hidden_layer_sizes=(num_features,), verbose=True, max_iter=int(1e3))
    pipe = Pipeline(steps=[('scale', MinMaxScaler()), ('nn', nn_min_max)])
    
    grid_search_nn = GridSearchCV(estimator=pipe, param_grid=param_space_nn, cv=StratifiedKFold(n_splits=10))
    
    nn_standard = MLPClassifier(hidden_layer_sizes=(num_features,), verbose=True, max_iter=int(1e3))
    pipe_standard = Pipeline(steps=[('scale', StandardScaler()), ('nn', nn_standard)])
    
    grid_search_nn_standard = GridSearchCV(estimator=pipe_standard, param_grid=param_space_nn, cv=StratifiedKFold(n_splits=10))
    
    nns.append((poss[0], poss[1], grid_search_nn, poss[2] + ' Min Max'))
    nns.append((poss[0], poss[1], grid_search_nn_standard, poss[2] + ' Standard'))
    
for X, y, nn, _ in nns:
    nn.fit(X, y)
    
for X, y, nn, label in nns:
    print (label)
    print ('{:.4f}'.format(logreg.score(X, y)))

Iteration 1, loss = 0.99679949
Iteration 2, loss = 0.69336787
Iteration 3, loss = 0.63696748
Iteration 4, loss = 0.62494847
Iteration 5, loss = 0.59357653
Iteration 6, loss = 0.58582651
Iteration 7, loss = 0.57128603
Iteration 8, loss = 0.56406300
Iteration 9, loss = 0.55771470
Iteration 10, loss = 0.53816465
Iteration 11, loss = 0.53819618
Iteration 12, loss = 0.53073269
Iteration 13, loss = 0.52082358
Iteration 14, loss = 0.51415449
Iteration 15, loss = 0.51183202
Iteration 16, loss = 0.50299021
Iteration 17, loss = 0.49812113
Iteration 18, loss = 0.48651988
Iteration 19, loss = 0.47992015
Iteration 20, loss = 0.48111291
Iteration 21, loss = 0.46525540
Iteration 22, loss = 0.46112939
Iteration 23, loss = 0.45301885
Iteration 24, loss = 0.44184863
Iteration 25, loss = 0.43388610
Iteration 26, loss = 0.42727217
Iteration 27, loss = 0.42396247
Iteration 28, loss = 0.41020301
Iteration 29, loss = 0.40377484
Iteration 30, loss = 0.40477592
Iteration 31, loss = 0.39158685
Iteration 32, los

Iteration 254, loss = 0.00692102
Iteration 255, loss = 0.00690603
Iteration 256, loss = 0.00667177
Iteration 257, loss = 0.00672188
Iteration 258, loss = 0.00677265
Iteration 259, loss = 0.00666687
Iteration 260, loss = 0.00658869
Iteration 261, loss = 0.00639447
Iteration 262, loss = 0.00646033
Iteration 263, loss = 0.00621229
Iteration 264, loss = 0.00611323
Iteration 265, loss = 0.00613388
Iteration 266, loss = 0.00617140
Iteration 267, loss = 0.00594079
Iteration 268, loss = 0.00598730
Iteration 269, loss = 0.00596502
Iteration 270, loss = 0.00584177
Iteration 271, loss = 0.00567066
Iteration 272, loss = 0.00587461
Iteration 273, loss = 0.00561742
Iteration 274, loss = 0.00567765
Iteration 275, loss = 0.00552899
Iteration 276, loss = 0.00545224
Iteration 277, loss = 0.00538433
Iteration 278, loss = 0.00535404
Iteration 279, loss = 0.00536733
Iteration 280, loss = 0.00525737
Iteration 281, loss = 0.00524896
Iteration 282, loss = 0.00518113
Training loss did not improve more than tol

Iteration 221, loss = 0.01031984
Iteration 222, loss = 0.01019511
Iteration 223, loss = 0.00998028
Iteration 224, loss = 0.00998338
Iteration 225, loss = 0.00974403
Iteration 226, loss = 0.00967363
Iteration 227, loss = 0.00984690
Iteration 228, loss = 0.00992522
Iteration 229, loss = 0.00959892
Iteration 230, loss = 0.00927099
Iteration 231, loss = 0.00901833
Iteration 232, loss = 0.00892239
Iteration 233, loss = 0.00900710
Iteration 234, loss = 0.00874799
Iteration 235, loss = 0.00871382
Iteration 236, loss = 0.00852084
Iteration 237, loss = 0.00844967
Iteration 238, loss = 0.00825572
Iteration 239, loss = 0.00826475
Iteration 240, loss = 0.00811035
Iteration 241, loss = 0.00804963
Iteration 242, loss = 0.00816108
Iteration 243, loss = 0.00827825
Iteration 244, loss = 0.00805948
Iteration 245, loss = 0.00789677
Iteration 246, loss = 0.00771386
Iteration 247, loss = 0.00753071
Iteration 248, loss = 0.00750387
Iteration 249, loss = 0.00738022
Iteration 250, loss = 0.00728991
Iteration 

Iteration 95, loss = 0.41123826
Iteration 96, loss = 0.40532598
Iteration 97, loss = 0.41660283
Iteration 98, loss = 0.39829156
Iteration 99, loss = 0.43074168
Iteration 100, loss = 0.39456349
Iteration 101, loss = 0.38628368
Iteration 102, loss = 0.38560152
Iteration 103, loss = 0.38479170
Iteration 104, loss = 0.43538878
Iteration 105, loss = 0.38898240
Iteration 106, loss = 0.39214348
Iteration 107, loss = 0.40519010
Iteration 108, loss = 0.71824287
Iteration 109, loss = 0.66750927
Iteration 110, loss = 0.43230835
Iteration 111, loss = 0.46573176
Iteration 112, loss = 0.43756301
Iteration 113, loss = 0.77234130
Iteration 114, loss = 0.59738195
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 1.05537415
Iteration 2, loss = 0.70583129
Iteration 3, loss = 0.68444552
Iteration 4, loss = 0.60652791
Iteration 5, loss = 0.64436769
Iteration 6, loss = 0.58096692
Iteration 7, loss = 0.59078721
Iteration 8, loss = 0.58609224
Iterati

Iteration 8, loss = 0.60820612
Iteration 9, loss = 0.73557372
Iteration 10, loss = 0.57350856
Iteration 11, loss = 0.58620927
Iteration 12, loss = 0.57351423
Iteration 13, loss = 0.65918335
Iteration 14, loss = 0.58504586
Iteration 15, loss = 0.63855433
Iteration 16, loss = 0.67435531
Iteration 17, loss = 0.58401106
Iteration 18, loss = 0.64511634
Iteration 19, loss = 0.60147589
Iteration 20, loss = 0.59889935
Iteration 21, loss = 0.64336986
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 1.02191933
Iteration 2, loss = 0.62995987
Iteration 3, loss = 0.63130978
Iteration 4, loss = 0.57860757
Iteration 5, loss = 0.60309136
Iteration 6, loss = 0.55989019
Iteration 7, loss = 0.64388797
Iteration 8, loss = 0.55495870
Iteration 9, loss = 0.63727168
Iteration 10, loss = 0.56073630
Iteration 11, loss = 0.62158834
Iteration 12, loss = 0.57224283
Iteration 13, loss = 0.59125387
Iteration 14, loss = 0.58375645
Iteration 15, loss = 0.56

...

Iteration 5, loss = 0.55547413
Iteration 6, loss = 0.54445210
Iteration 7, loss = 0.52670737
Iteration 8, loss = 0.51707451
Iteration 9, loss = 0.51182759
Iteration 10, loss = 0.53866777
Iteration 11, loss = 0.49828546
Iteration 12, loss = 0.47947046
Iteration 13, loss = 0.48165486
Iteration 14, loss = 0.47721297
Iteration 15, loss = 0.46718363
Iteration 16, loss = 0.46955456
Iteration 17, loss = 0.47121291
Iteration 18, loss = 0.43553389
Iteration 19, loss = 0.42886332
Iteration 20, loss = 0.41717343
Iteration 21, loss = 0.41443273
Iteration 22, loss = 0.40460256
Iteration 23, loss = 0.39906234
Iteration 24, loss = 0.42266476
Iteration 25, loss = 0.41475121
Iteration 26, loss = 0.38875205
Iteration 27, loss = 0.38068907
Iteration 28, loss = 0.36715510
Iteration 29, loss = 0.36520520
Iteration 30, loss = 0.36324136
Iteration 31, loss = 0.35380494
Iteration 32, loss = 0.36050887
Iteration 33, loss = 0.34698726
Iteration 34, loss = 0.35435978
Iteration 35, loss = 0.33693390
Iteration 36,

Iteration 257, loss = 0.00452787
Iteration 258, loss = 0.00439916
Iteration 259, loss = 0.00435659
Iteration 260, loss = 0.00484494
Iteration 261, loss = 0.00447481
Iteration 262, loss = 0.00418235
Iteration 263, loss = 0.00416337
Iteration 264, loss = 0.00407372
Iteration 265, loss = 0.00412732
Iteration 266, loss = 0.00422538
Iteration 267, loss = 0.00431326
Iteration 268, loss = 0.00426203
Iteration 269, loss = 0.00419599
Iteration 270, loss = 0.00385062
Iteration 271, loss = 0.00387557
Iteration 272, loss = 0.00378903
Iteration 273, loss = 0.00381323
Iteration 274, loss = 0.00404019
Iteration 275, loss = 0.00376207
Iteration 276, loss = 0.00359831
Iteration 277, loss = 0.00354320
Iteration 278, loss = 0.00352426
Iteration 279, loss = 0.00348108
Iteration 280, loss = 0.00344113
Iteration 281, loss = 0.00372768
Iteration 282, loss = 0.00407411
Iteration 283, loss = 0.00401664
Iteration 284, loss = 0.00331797
Iteration 285, loss = 0.00349082
Iteration 286, loss = 0.00328310
Iteration 

Iteration 40, loss = 0.00644388
Iteration 41, loss = 0.00655870
Iteration 42, loss = 0.00639294
Iteration 43, loss = 0.00564025
Iteration 44, loss = 0.00527733
Iteration 45, loss = 0.00510058
Iteration 46, loss = 0.00529705
Iteration 47, loss = 0.00489943
Iteration 48, loss = 0.00551608
Iteration 49, loss = 0.00684527
Iteration 50, loss = 0.00553487
Iteration 51, loss = 0.00467682
Iteration 52, loss = 0.01097890
Iteration 53, loss = 0.00932079
Iteration 54, loss = 0.00633960
Iteration 55, loss = 0.00428345
Iteration 56, loss = 0.00466376
Iteration 57, loss = 0.00359393
Iteration 58, loss = 0.00330724
Iteration 59, loss = 0.00339694
Iteration 60, loss = 0.00327914
Iteration 61, loss = 0.00293074
Iteration 62, loss = 0.00274419
Iteration 63, loss = 0.00262756
Iteration 64, loss = 0.00253446
Iteration 65, loss = 0.00245625
Iteration 66, loss = 0.00257679
Iteration 67, loss = 0.00308567
Iteration 68, loss = 0.00264558
Iteration 69, loss = 0.00240743
Iteration 70, loss = 0.00232361
Iteratio

Iteration 53, loss = 0.00145576
Iteration 54, loss = 0.00144371
Iteration 55, loss = 0.00139399
Iteration 56, loss = 0.00134373
Iteration 57, loss = 0.00130052
Iteration 58, loss = 0.00125804
Iteration 59, loss = 0.00122077
Iteration 60, loss = 0.00121178
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.73063834
Iteration 2, loss = 0.39514126
Iteration 3, loss = 0.28186424
Iteration 4, loss = 0.22653758
Iteration 5, loss = 0.16834701
Iteration 6, loss = 0.18275288
Iteration 7, loss = 0.14691324
Iteration 8, loss = 0.12763235
Iteration 9, loss = 0.10835671
Iteration 10, loss = 0.08845603
Iteration 11, loss = 0.06009020
Iteration 12, loss = 0.04364210
Iteration 13, loss = 0.03327953
Iteration 14, loss = 0.02671119
Iteration 15, loss = 0.02305254
Iteration 16, loss = 0.02061284
Iteration 17, loss = 0.01788743
Iteration 18, loss = 0.01689848
Iteration 19, loss = 0.01466075
Iteration 20, loss = 0.01300497
Iteration 21, loss = 0.

Iteration 53, loss = 0.00600235
Iteration 54, loss = 0.00599021
Iteration 55, loss = 0.00547526
Iteration 56, loss = 0.00517031
Iteration 57, loss = 0.00489582
Iteration 58, loss = 0.00475875
Iteration 59, loss = 0.00472210
Iteration 60, loss = 0.00456251
Iteration 61, loss = 0.00438866
Iteration 62, loss = 0.00425829
Iteration 63, loss = 0.00424565
Iteration 64, loss = 0.00427914
Iteration 65, loss = 0.00403903
Iteration 66, loss = 0.00380603
Iteration 67, loss = 0.00361380
Iteration 68, loss = 0.00355617
Iteration 69, loss = 0.00351490
Iteration 70, loss = 0.00337370
Iteration 71, loss = 0.00324718
Iteration 72, loss = 0.00316479
Iteration 73, loss = 0.00309272
Iteration 74, loss = 0.00305521
Iteration 75, loss = 0.00334229
Iteration 76, loss = 0.00324776
Iteration 77, loss = 0.00299350
Iteration 78, loss = 0.00286522
Iteration 79, loss = 0.00276166
Iteration 80, loss = 0.00269187
Iteration 81, loss = 0.00265420
Iteration 82, loss = 0.00261205
Iteration 83, loss = 0.00251353
Iteratio

In [137]:
for X, y, nn, label in nns:
    print (label)
    y_pred = nn.predict(X)
    print (confusion_matrix(y, y_pred))

Data Min Max
[[329   0]
 [  0 783]]
Data Standard
[[329   0]
 [  0 783]]
Data Invert Min Max
[[1112    0]
 [   0 1112]]
Data Invert Standard
[[1112    0]
 [   0 1112]]


## Predict Scores - Regression

In [51]:
%%time

## Random Forest
param_space_score_random_forest = {
#     'multi__estimator__n_estimators' : [100, 1000],
#     'multi__estimator__criterion' : ['mse', 'mae'],
#     'multi__estimator__bootstrap' : [True, False]
}

random_score_forests = []
for poss in sklearn_score_space:
    random_forest_min_max = RandomForestRegressor(n_jobs=-1)
    multi_output_min_max = MultiOutputRegressor(random_forest_min_max, n_jobs=-1)
    
    pipe = Pipeline(steps=[('scale', MinMaxScaler()),
                           ('multi', multi_output_min_max)])
    
    grid_search_forest = GridSearchCV(estimator=pipe, 
                                      param_grid=param_space_score_random_forest, 
                                      cv=10)
    
    random_forest_standard = RandomForestRegressor(n_jobs=-1)
    multi_output_standard = MultiOutputRegressor(random_forest_standard, n_jobs=-1)
    
    pipe_standard = Pipeline(steps=[('scale', StandardScaler()),
                                    ('multi', multi_output_standard)])
    
    grid_search_forest_standard = GridSearchCV(estimator=pipe_standard, 
                                               param_grid=param_space_score_random_forest, 
                                               cv=10)
    
    random_score_forests.append((poss[0], poss[1], grid_search_forest, poss[2] + ' Min Max'))
    random_score_forests.append((poss[0], poss[1], grid_search_forest_standard, poss[2] + ' Standard'))

for X, y, forest, _ in random_score_forests:
    forest.fit(X, y)
    
for X, y, forest, label in random_score_forests:
    print (label)
    # this returns R^2 (not accuracy)
    print ('{:.4f}'.format(forest.score(X, y)))

Data Min Max
0.8778
Data Standard
0.8765
Data Invert Min Max
0.8886
Data Invert Standard
0.8877
CPU times: user 6.48 s, sys: 4.98 s, total: 11.5 s
Wall time: 23min 13s


In [52]:
for _, _, forest, label in random_score_forests:
    print (label)
    print (forest.best_params_)

Data Min Max
{}
Data Standard
{}
Data Invert Min Max
{}
Data Invert Standard
{}


In [53]:
for _, _, forest, label in random_score_forests:
    print (label)
    print (forest.predict(X=row))
    print (forest.predict(X=row_unc))

Data Min Max
[[63.81 71.84]]
[[69.11 71.06]]
Data Standard
[[62.25 70.08]]
[[68.02 71.18]]
Data Invert Min Max
[[60.24 67.59]]
[[67.72 70.53]]
Data Invert Standard
[[63.59 67.43]]
[[66.74 69.13]]


### Linear Regression

In [48]:
%%time

## Random Forest
param_space_score_linear_regression = {
    'multi__estimator__fit_intercept' : [True, False]
}

score_linear_regressions = []
for poss in sklearn_score_space:
    linear_none = LinearRegression()
    multi_none = MultiOutputRegressor(linear_none, n_jobs=-1)
    pipe_none = Pipeline(steps=[('multi', multi_none)])
    grid_search_none = GridSearchCV(estimator=pipe_none,
                                   param_grid=param_space_score_linear_regression,
                                   cv=10)
    
    linear_min_max = LinearRegression()
    multi_output_min_max = MultiOutputRegressor(linear_min_max, n_jobs=-1)
    
    pipe = Pipeline(steps=[('scale', MinMaxScaler()),
                           ('multi', multi_output_min_max)])
    
    grid_search_linear = GridSearchCV(estimator=pipe, 
                                      param_grid=param_space_score_linear_regression, 
                                      cv=10)
    
    linear_standard = LinearRegression()
    multi_output_standard = MultiOutputRegressor(linear_standard, n_jobs=-1)
    
    pipe_standard = Pipeline(steps=[('scale', StandardScaler()),
                                    ('multi', multi_output_standard)])
    
    grid_search_linear_standard = GridSearchCV(estimator=pipe_standard, 
                                               param_grid=param_space_score_linear_regression, 
                                               cv=10)
    
    score_linear_regressions.append((poss[0], poss[1], grid_search_none, poss[2] + ' None'))
    score_linear_regressions.append((poss[0], poss[1], grid_search_linear, poss[2] + ' Min Max'))
    score_linear_regressions.append((poss[0], poss[1], grid_search_linear_standard, poss[2] + ' Standard'))

for X, y, lin, _ in score_linear_regressions:
    lin.fit(X, y)
    
for X, y, lin, label in score_linear_regressions:
    print (label)
    # this returns R^2 (not accuracy)
    print ('{:.4f}'.format(lin.score(X, y)))

Data None
0.8458
Data Min Max
0.8458
Data Standard
0.8458
Data Invert None
0.5756
Data Invert Min Max
0.5748
Data Invert Standard
0.5755
CPU times: user 3.73 s, sys: 10.5 s, total: 14.2 s
Wall time: 1min 58s


In [49]:
for _, _, lin, label in score_linear_regressions:
    print (label)
    print (lin.best_params_)

Data None
{'multi__estimator__fit_intercept': True}
Data Min Max
{'multi__estimator__fit_intercept': False}
Data Standard
{'multi__estimator__fit_intercept': True}
Data Invert None
{'multi__estimator__fit_intercept': True}
Data Invert Min Max
{'multi__estimator__fit_intercept': False}
Data Invert Standard
{'multi__estimator__fit_intercept': True}


In [50]:
for _, _, lin, label in score_linear_regressions:
    print (label)
    print (lin.predict(X=row))
    print (lin.predict(X=row_unc))

Data None
[[58.44418803 71.07457488]]
[[73.13338337 67.15193635]]
Data Min Max
[[58.44418803 71.07457488]]
[[73.13338337 67.15193635]]
Data Standard
[[58.44418803 71.07457488]]
[[73.13338337 67.15193635]]
Data Invert None
[[57.13170851 44.14608095]]
[[74.74826886 74.63820606]]
Data Invert Min Max
[[57.3678851  44.38225753]]
[[74.73944021 74.6293774 ]]
Data Invert Standard
[[57.13319458 44.1465042 ]]
[[74.7504211  74.66476279]]


### Ridge, Lasso, ElasticNet

In [54]:
%%time

## Random Forest
param_space_score_lasso = {
    'multi__estimator__alpha' : [10**i for i in range(-5, 2) if i != 0],
    'multi__estimator__positive' : [True, False],
    'multi__estimator__fit_intercept' : [True, False]
}

score_lassos = []
for poss in sklearn_score_space:
    lasso_min_max = Lasso()
    multi_output_min_max = MultiOutputRegressor(lasso_min_max, n_jobs=-1)
    
    pipe = Pipeline(steps=[('scale', MinMaxScaler()),
                           ('multi', multi_output_min_max)])
    
    grid_search_lasso = GridSearchCV(estimator=pipe, 
                                      param_grid=param_space_score_lasso, 
                                      cv=10)
    
    lasso_standard = Lasso()
    multi_output_standard = MultiOutputRegressor(lasso_standard, n_jobs=-1)
    
    pipe_standard = Pipeline(steps=[('scale', StandardScaler()),
                                    ('multi', multi_output_standard)])
    
    grid_search_lasso_standard = GridSearchCV(estimator=pipe_standard, 
                                               param_grid=param_space_score_lasso, 
                                               cv=10)
    
    score_lassos.append((poss[0], poss[1], grid_search_lasso, poss[2] + ' Min Max'))
    score_lassos.append((poss[0], poss[1], grid_search_lasso_standard, poss[2] + ' Standard'))

for X, y, lin, _ in score_lassos:
    lin.fit(X, y)
    
for X, y, lin, label in score_lassos:
    print (label)
    # this returns R^2 (not accuracy)
    print ('{:.4f}'.format(lin.score(X, y)))

Data Min Max
0.2617
Data Standard
0.3156
Data Invert Min Max
0.3015
Data Invert Standard
0.3210
CPU times: user 32.2 s, sys: 24.8 s, total: 57 s
Wall time: 16min 50s


In [55]:
for _, _, lin, label in score_lassos:
    print (label)
    print (lin.best_params_)

Data Min Max
{'multi__estimator__alpha': 0.1, 'multi__estimator__fit_intercept': True, 'multi__estimator__positive': True}
Data Standard
{'multi__estimator__alpha': 0.1, 'multi__estimator__fit_intercept': True, 'multi__estimator__positive': True}
Data Invert Min Max
{'multi__estimator__alpha': 0.1, 'multi__estimator__fit_intercept': False, 'multi__estimator__positive': True}
Data Invert Standard
{'multi__estimator__alpha': 0.1, 'multi__estimator__fit_intercept': True, 'multi__estimator__positive': True}


In [56]:
%%time

## Random Forest
param_space_score_ridge = {
    'multi__estimator__alpha' : [10**i for i in range(-5, 5) if i != 0],
    'multi__estimator__fit_intercept' : [True, False]
}

score_ridges = []
for poss in sklearn_score_space:
    ridge_min_max = Ridge()
    multi_output_min_max = MultiOutputRegressor(ridge_min_max, n_jobs=-1)
    
    pipe = Pipeline(steps=[('scale', MinMaxScaler()),
                           ('multi', multi_output_min_max)])
    
    grid_search_ridge = GridSearchCV(estimator=pipe, 
                                      param_grid=param_space_score_ridge, 
                                      cv=10)
    
    ridge_standard = Ridge()
    multi_output_standard = MultiOutputRegressor(ridge_standard, n_jobs=-1)
    
    pipe_standard = Pipeline(steps=[('scale', StandardScaler()),
                                    ('multi', multi_output_standard)])
    
    grid_search_ridge_standard = GridSearchCV(estimator=pipe_standard, 
                                               param_grid=param_space_score_ridge, 
                                               cv=10)
    
    score_ridges.append((poss[0], poss[1], grid_search_ridge, poss[2] + ' Min Max'))
    score_ridges.append((poss[0], poss[1], grid_search_ridge_standard, poss[2] + ' Standard'))

for X, y, lin, _ in score_ridges:
    lin.fit(X, y)
    
for X, y, lin, label in score_ridges:
    print (label)
    # this returns R^2 (not accuracy)
    print ('{:.4f}'.format(lin.score(X, y)))

Data Min Max
0.3259
Data Standard
0.4517
Data Invert Min Max
0.3417
Data Invert Standard
0.4275
CPU times: user 21.2 s, sys: 12.4 s, total: 33.6 s
Wall time: 1min 48s


In [57]:
for _, _, lin, label in score_ridges:
    print (label)
    print (lin.best_params_)

Data Min Max
{'multi__estimator__alpha': 100, 'multi__estimator__fit_intercept': True}
Data Standard
{'multi__estimator__alpha': 1000, 'multi__estimator__fit_intercept': True}
Data Invert Min Max
{'multi__estimator__alpha': 100, 'multi__estimator__fit_intercept': True}
Data Invert Standard
{'multi__estimator__alpha': 1000, 'multi__estimator__fit_intercept': True}


In [61]:
%%time

## Random Forest
param_space_score_elastic = {
    'multi__estimator__alpha' : [10**i for i in range(-5, 5) if i != 0],
    'multi__estimator__l1_ratio' : np.linspace(0, 1, num=5)
}

score_elastic_nets = []
for poss in sklearn_score_space:
    elastic_min_max = ElasticNet()
    multi_output_min_max = MultiOutputRegressor(elastic_min_max, n_jobs=-1)
    
    pipe = Pipeline(steps=[('scale', MinMaxScaler()),
                           ('multi', multi_output_min_max)])
    
    grid_search_elastic = GridSearchCV(estimator=pipe, 
                                      param_grid=param_space_score_elastic, 
                                      cv=10)
    
    elastic_standard = ElasticNet()
    multi_output_standard = MultiOutputRegressor(elastic_standard, n_jobs=-1)
    
    pipe_standard = Pipeline(steps=[('scale', StandardScaler()),
                                    ('multi', multi_output_standard)])
    
    grid_search_elastic_standard = GridSearchCV(estimator=pipe_standard, 
                                               param_grid=param_space_score_elastic, 
                                               cv=10)
    
    score_elastic_nets.append((poss[0], poss[1], grid_search_elastic, poss[2] + ' Min Max'))
    score_elastic_nets.append((poss[0], poss[1], grid_search_elastic_standard, poss[2] + ' Standard'))

for X, y, lin, _ in score_elastic_nets:
    lin.fit(X, y)
    
for X, y, lin, label in score_elastic_nets:
    print (label)
    # this returns R^2 (not accuracy)
    print ('{:.4f}'.format(lin.score(X, y)))

Data Min Max
0.2820
Data Standard
0.2200
Data Invert Min Max
0.2961
Data Invert Standard
0.4257
CPU times: user 1min, sys: 42.4 s, total: 1min 43s
Wall time: 33min 22s


In [131]:
for _, _, lin, label in score_elastic_nets:
    print (label)
    print (lin.best_params_)

Data Min Max
{'multi__estimator__alpha': 0.1}
Data Standard
{'multi__estimator__alpha': 10}
Data Invert Min Max
{'multi__estimator__alpha': 0.1}
Data Invert Standard
{'multi__estimator__alpha': 0.1}


In [60]:
np.linspace(0, 1, num=5)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])